In [1]:
import pandas as pd
import re
from llama_index.core import Document
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from transformers import BitsAndBytesConfig
from llama_index.core import VectorStoreIndex
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core import Settings
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.retrievers import BaseRetriever
from llama_index.core import get_response_synthesizer
from llama_index.core.response_synthesizers import BaseSynthesizer
from llama_index.core import PromptTemplate
import torch

In [2]:
def create_docs(df):
    documents = []
    for i, row in df.iterrows():
        metadata = {
            "ссылка": row.url,
            "заголовок": row.title_preview,
            "описание": row.description_preview,
            "цена": row.price,
            "длительность": row.duration_months,
            "инструменты": ", ".join(row.technologies)}
        documents.append(Document(text=row.description, metadata=metadata,
                         excluded_embed_metadata_keys=["ссылка", "цена", "длительность"]))

    return documents


df = pd.read_json("data/courses_plus.jsonl", lines=True)
df.dropna(inplace=True)
df.drop_duplicates(subset=["title_preview"], inplace=True)
df.title_preview = df.title_preview.apply(lambda x: x.split("\n")[0])
df.reset_index(drop=True, inplace=True)
df.head()

,url,title_preview,description_preview,duration_months,payment_duration_months,title,price,description,technologies,program,has_none_values
0,https://gb.ru/s/ai-integration-specialist,Специалист по внедрению Искусственного Интеллекта,Лёгкий путь в Machine Learning и Data Science....,6,36,Стань специалистом по Искусственному Интеллект...,3599,Создавай AI-продукты и зарабатывай на их внедр...,"[Lean Canvas, JTBD, Retention, Figma, CustDev,...",Создание GPT-агентов | Модуль 16 недель | В эт...,False
1,https://gb.ru/geek_university/developer,Разработчик: старт в ИТ с нуля до Junior,"Идеальная программа для тех, кто хочет попасть...",12,36,Разработчик: старт в ИТ с нуля до Junior,3772,Станьте востребованным разработчиком. Вы изучи...,"[Программирование, Продукт и проекты, Тестиров...",Основной блок\nПогружение в сферу ИТ. Вы изучи...,False
2,https://gb.ru/geek_university/developer/progra...,Python-разработчик: быстрый старт в профессии,Научитесь использовать язык программирования P...,9,36,Python-разработчик: быстрый старт в профессии,4049,• Изучите Python с нуля и без навыков программ...,"[Программирование, Python-разработчик]",Программа обучения\nСпециалист\nPython-разрабо...,False
3,https://gb.ru/geek_university/developer/progra...,Программист с нуля до Junior,Выберите профессию в программировании в процес...,12,36,Программист с нуля до Junior,3577,Станьте востребованным инженером-программистом...,"[Программирование, Инженер-программист]",Основной блок\nПогружение в сферу ИТ. Вы изучи...,False
4,https://gb.ru/geek_university/developer/qa-eng...,Специалист по автоматизированному тестированию,Вы изучите технологии автоматизированного тест...,9,36,Специалист по автоматизированному тестированию,3201,Получите востребованную профессию инженера по ...,"[Специалист по автоматизации тестирования, Тес...",Программа обучения\nСпециалист\nАвтоматизация ...,False


In [3]:
docs = create_docs(df)

In [4]:
def setup_llm():
    system_prompt = "Ты AI ассистент, который на основе описания вакансии рекомендует курсы из контекста."
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )
    llm = HuggingFaceLLM(
        model_name="mistral_model",
        tokenizer_name="mistral_model",
        query_wrapper_prompt=PromptTemplate(
            "<s>[INST] {query_str} [/INST] </s>\n"),
        context_window=8192,
        max_new_tokens=512,
        model_kwargs={"quantization_config": quantization_config},
        # tokenizer_kwargs={},
        generate_kwargs={"temperature": 0},
        device_map="auto",
        system_prompt=system_prompt,
    )
    return llm

In [5]:
llm = setup_llm()
Settings.llm = llm
Settings.embed_model = HuggingFaceEmbedding(
    model_name="intfloat/multilingual-e5-base"
)

index = VectorStoreIndex.from_documents(documents=docs, show_progress=True,)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/92 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/92 [00:00<?, ?it/s]

In [6]:
retriever = index.as_retriever(similarity_top_k=7, node_postprocessors=[
                               SimilarityPostprocessor(similarity_cutoff=0.85)])

In [7]:
stack_prompt = PromptTemplate(
    "У тебя есть описание вакансии: {query_str}\n"
    "Тебе нужно найти инструменты из описания вакансии и написать их на русском языке в виде строки с разделителем запятой и без повторений"
)

answer_prompt = PromptTemplate(
    "Тебе дано 2 множества:\n"
    "Первое множество: {vac_stack}\n"
    "Второе множество: {cour_stack}.\n"
    "Найди пересечение этих множеств и напиши их на русском языке в виде строки с разделителем запятой и без повторений"
)


class RAGStringQueryEngine(CustomQueryEngine):
    """RAG String Query Engine."""

    retriever: BaseRetriever
    response_synthesizer: BaseSynthesizer
    llm: HuggingFaceLLM
    stack_prompt: PromptTemplate
    answer_prompt: PromptTemplate

    def custom_query(self, query_str: str):
        query_str = re.sub("\n", " ", query_str)
        query_str = re.sub(" +", " ", query_str)
        query_str = query_str.strip()
        nodes = self.retriever.retrieve(query_str)
        stack = str(self.llm.complete(stack_prompt.format(
            query_str=query_str))).strip().split(",")
        stack = [el.strip() for el in stack]
        responses = []
        for n in nodes:
            course = {
                "title": n.node.metadata["заголовок"],
                "price": n.node.metadata["цена"],
                "link": n.node.metadata["ссылка"],
                "desc": n.node.metadata["описание"],
                "vac_stack": stack,
                "match": float(n.score),
                "duration": n.node.metadata["длительность"]
            }
            # print(answer_prompt.format(vac_stack=stack, cour_stack=n.node.metadata["инструменты"]))
            response = self.llm.complete(
                answer_prompt.format(
                    vac_stack=stack, cour_stack=n.node.metadata["инструменты"])
            )
            response = str(response).strip().split(",")
            response = [el.strip() for el in response]
            course["cover"] = response
            responses.append(course)

        return responses

In [8]:
synthesizer = get_response_synthesizer(response_mode="compact")

query_engine = RAGStringQueryEngine(
    retriever=retriever,
    response_synthesizer=synthesizer,
    llm=llm,
    stack_prompt=stack_prompt,
    answer_prompt=answer_prompt,
)

In [9]:
vacancy = """
Lamoda Group – это крупнейшая в России и СНГ онлайн-платформа по продаже fashion & lifestyle товаров.

В настоящий момент мы ищем коллегу на позицию Python/Go разработчика в команду Marketing Development, где ребята занимаются разными системами коммуникации с клиентами, а также множеством других сервисов, направленных на упрощение работы команды маркетинга :)


Готовы рассматривать кандидатов без опыта с Golang, но обязательно с желанием развиваться - у нас есть своя выстроенная система онбординга и обучения!

Как мы работаем:

пишем на последних версиях Python/Go;
используем NSQ для внутренних очередей, а Apache Kafka для межсервисного взаимодействия;
PostgreSQL в качестве основной СУБД, Redis, Aerospike;
все приложения упакованы в Docker и деплоятся нажатием кнопки;
у нас отличный CI/CD, запускаем тесты, security & style checks на каждый pull request;
все pull request проходят 360 code review;
участвуем в разработке и проектировании архитектурных решений, а также в реализации бизнес решений.
Чем предстоит заниматься:

писать новые и развивать существующие сервисы на Python и Go;
участвовать в анализе, проектировании и оценке технических решений вместе с командой;
писать unit и функциональные тесты;
участвовать в code review.
Мы ожидаем:

опыт коммерческой разработки бэкенда на Python от 2-х лет, опыт на Golang будет плюсом;
опыт работы с SQL базами данных (на уровне join-ов, оптимизации запросов);
умение проектировать API сервисов;
опыт работы с нереляционными базами данных (Redis);
опыт работы с Docker, k8s и Git.
"""

In [10]:
response = query_engine.query(vacancy)

/home/pc/Desktop/cp-cfo-2024/venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [11]:
print(response[0])

{'title': 'Backend-разработчик', 'price': 4049, 'link': 'https://gb.ru/geek_university/developer/programmer/backend', 'desc': 'Вы научитесь создавать внутреннюю систему сайтов: выстраивать обмен данными, работать с базами данных, обеспечивать бесперебойную и производительную работу системы', 'vac_stack': ['Python/Go разработчик', 'Marketing Development', 'NSQ', 'Apache Kafka', 'PostgreSQL', 'Redis', 'Aerospike', 'Docker', 'CI/CD', 'unit/functional tests', 'code review', 'SQL', 'API', 'non-relational databases', 'Docker', 'k8s', 'Git.'], 'match': 0.8715546317208881, 'duration': 9, 'cover': ["```python\nset1 = ['Python/Go разработчик'", "'Marketing Development'", "'NSQ'", "'Apache Kafka'", "'PostgreSQL'", "'Redis'", "'Aerospike'", "'Docker'", "'CI/CD'", "'unit/functional tests'", "'code review'", "'SQL'", "'API'", "'non-relational databases'", "'Docker'", "'k8s'", "'Git.']\nset2 = ['Освоите 17 инструментов Python-разработчика", 'Среды разработки', 'PyCharm', 'Docker', 'Linux', 'Kubernete

In [12]:
len(response)

7